In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import backend as K
from tqdm import tqdm
from dataprep import transform_indices
from dataprep import transform_indices, reindex_data

2023-03-29 18:45:31.516902: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-29 18:45:31.719681: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-03-29 18:45:32.383723: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-03-29 18:45:32.383804: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinf

In [2]:
# ------------------------- data -----------------------------------------------------
data_train = pd.read_csv('training.csv')
data_test = pd.read_csv('testset.csv')

# from kaggle siminar
def transform_data(train, test, holdout=None, userid='userid', itemid='movieid'):
    train_new, data_index = transform_indices(train, userid, itemid)
    testset_new = reindex_data(test, data_index, fields='items')
    if holdout is None:
        return train_new, testset_new, data_index
    holdout_new = reindex_data(holdout, data_index, fields='items')
    return train_new, testset_new, holdout_new, data_index

training, testset, data_index_source = transform_data(
    data_train, data_test, userid = 'userid', itemid='movieid'
)
# --------------------------------------------------------------------------------------
print(data_test['userid'].nunique(), data_test['movieid'].nunique())
print(testset['userid'].nunique(), testset['movieid'].nunique())


2963 17102
2963 16200


In [3]:
data_description_source = dict(
    users = data_index_source['users'].name,
    items = data_index_source['items'].name,
    feedback = 'rating',
    n_users = len(data_index_source['users']),
    n_items = len(data_index_source['items']), 
)

In [4]:
data_description_source

{'users': 'userid',
 'items': 'movieid',
 'feedback': 'rating',
 'n_users': 71297,
 'n_items': 17092}

In [5]:
df = testset.sort_values(['userid', 'timestamp'], ascending=[True, True]).reset_index()
user_avg_rating = df.groupby(df.userid)['rating'].expanding().mean().reset_index()
user_std_rating = df.groupby(df.userid)['rating'].expanding().std().reset_index()
user_rating_count = df.groupby(df.userid)['rating'].expanding().count().reset_index()

df['user_avg_rating'] = user_avg_rating['rating']
df['user_std_rating'] = user_std_rating['rating'].fillna(0)
df['user_rating_count'] = user_rating_count['rating']

test_data = df.copy()
test_data.shape

(1368134, 8)

In [6]:
test_data.head(3)

,index,userid,movieid,rating,timestamp,user_avg_rating,user_std_rating,user_rating_count
0,693,55,35,3.0,1274766552,3.0,0.000000,1.0
1,52,55,471,1.0,1274766571,2.0,1.414214,2.0
2,517,55,834,2.0,1274766574,2.0,1.000000,3.0


In [7]:
# ---------------------- Check NaN --------------------------
nan_cols = test_data.columns[test_data.isnull().any()].tolist()
nan_cols

[]

In [8]:
# ------------------- Normalize numerical columns -----------------------------------------------------
numerical_cols = ['user_avg_rating', 'user_std_rating',  'user_rating_count']
for col in numerical_cols:
    test_data[col] = (test_data[col] - test_data[col].min()) / (test_data[col].max() - test_data[col].min())   

In [9]:
# ----------------------- Load models ---------------------------------------
tf.config.run_functions_eagerly(True)

neuralCF = tf.keras.models.load_model('neuralCF_2_crossE')
neuralCF.compile(
    loss = tf.keras.losses.BinaryCrossentropy(from_logits=True),
    optimizer = 'adam',
    run_eagerly=True
)

deepfm = tf.keras.models.load_model('deepfm_2_crossE')
deepfm.compile(
    loss = tf.keras.losses.BinaryCrossentropy(from_logits=True),
    optimizer = 'adam', 
    run_eagerly = True
)

2023-03-29 18:45:39.658929: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-03-29 18:45:39.658986: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2023-03-29 18:45:39.659027: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:163] no NVIDIA GPU device is present: /dev/nvidia0 does not exist
2023-03-29 18:45:39.659528: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [10]:
movie_embedding = test_data.iloc[test_data.movieid.drop_duplicates().index]['movieid'].reset_index(drop=True)

In [11]:
top_50_recs = pd.read_csv('pd_top_50_recs_coffee.csv').drop(columns=['Unnamed: 0'])
top_50_recs = top_50_recs.T.reset_index(drop=False)
top_50_recs = top_50_recs.rename(columns={'index': 'userid'})
top_50_recs

,userid,0,1,2,3,4,5,6,7,8,...,40,41,42,43,44,45,46,47,48,49
0,55,206,106,882,225,178,421,62,767,165,...,214,1230,965,880,791,790,1224,1410,204,864
1,133,146,262,226,179,221,162,101,491,215,...,635,592,42,117,280,310,73,634,165,577
2,136,106,110,376,165,216,421,62,410,577,...,284,510,428,472,222,220,965,225,867,162
3,213,671,530,791,565,774,421,658,674,770,...,621,578,968,168,2353,762,786,768,1373,966
4,539,732,791,595,711,621,216,965,530,578,...,198,76,750,206,416,804,214,225,220,968
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2958,282663,711,864,865,668,671,867,732,774,617,...,1306,767,2576,777,218,566,1310,883,219,780
2959,282836,421,106,165,61,770,206,110,62,410,...,73,226,750,674,214,156,773,220,968,230
2960,282999,1037,110,216,637,711,42,767,864,867,...,121,1545,883,154,863,218,165,764,595,1366
2961,283047,965,966,968,756,793,1850,961,1520,799,...,1289,2242,123,856,789,1068,967,1054,712,161


In [12]:
def get_top50_by_embed(userid):  
    return set(top_50_recs.loc[top_50_recs['userid'] == str(userid)][0:].values.tolist()[0][1:])

In [13]:
features = ['userid', 'movieid', 'rating', 'user_avg_rating', 'user_std_rating', 'user_rating_count', 'timestamp']

In [14]:
test_timepoint = test_data['timestamp'].quantile(q=0.80, interpolation='nearest')

In [15]:
data = test_data.copy()
unique_ids = data['userid'].unique()
random_timestamps = []

for id in unique_ids:
    timestamps = data[data['userid'] == id]['timestamp']
    random_timestamps.append(test_timepoint)

new_df = pd.DataFrame({'userid': unique_ids,
                       'timestamp': random_timestamps})

new_df.head(3)

,userid,timestamp
0,55,1336846537
1,133,1336846537
2,136,1336846537


In [16]:
movie_recommendations = []

for target_userid in tqdm(new_df['userid'].tolist()):
    
    target_timestamp = new_df[new_df['userid'] == target_userid]['timestamp'].iloc[0]
    timestamp_sensitive_features_user = ['user_avg_rating', 'user_std_rating', 'user_rating_count']  

    timestamp_insensitive_features_user = ['userid']
    timestamp_insensitive_features_movie = ['movieid']
    
    movie_retrieval_candidates = data[data.timestamp <=target_timestamp][timestamp_insensitive_features_movie].drop_duplicates()
    
    user_ts_insensitive_records = data[data.userid == target_userid][timestamp_insensitive_features_user].drop_duplicates().to_dict('records')[0]
    
    for f in timestamp_insensitive_features_user:
        movie_retrieval_candidates[f] = user_ts_insensitive_records[f]

    user_ts_sensitive_records = data[data.userid == target_userid][timestamp_sensitive_features_user].drop_duplicates().to_dict('records')[0]
    for f in timestamp_sensitive_features_user:
        movie_retrieval_candidates[f] = user_ts_sensitive_records[f]
 
    user_history = data[(data.userid == target_userid) & (data.timestamp <= target_timestamp)]

    movie_retrieval_candidates = movie_retrieval_candidates[~movie_retrieval_candidates.movieid.isin(user_history.movieid)]
    retrieval_ds = tf.data.Dataset.from_tensor_slices((dict(movie_retrieval_candidates))).batch(32)
    
    # Get top 100 from neuralCF
    retrieval_result = np.concatenate(neuralCF.predict(retrieval_ds)) 
    top100_indexes = np.argpartition(retrieval_result, -100)[-100:]
    movie_ranking_candidates = movie_retrieval_candidates.iloc[top100_indexes]
    
    # Get top 50 from folding-in (warm-start scenerio)
    top_50_folding_in = get_top50_by_embed(target_userid)
        
    movie_retrieval_via_embedding = movie_retrieval_candidates[(movie_retrieval_candidates.movieid.isin(top_50_folding_in)) 
                                    &
                                    (~movie_retrieval_candidates.movieid.isin(movie_ranking_candidates.movieid))]
    
    # Concat recommended movies from folding-in and neuralCF
    movie_ranking_candidates = pd.concat([movie_retrieval_via_embedding, movie_ranking_candidates])
    ranking_ds = tf.data.Dataset.from_tensor_slices((dict(movie_ranking_candidates))).batch(32)
    
    # Predict recommendation and get top-20
    ranking_result = np.concatenate(deepfm.predict(ranking_ds))
    top20_indexes = np.argpartition(ranking_result, -20)[-20:]
    top20_movieid = movie_ranking_candidates.iloc[top20_indexes].movieid
    movie_recommendations.append(list(top20_movieid))
    

  0%|          | 0/2963 [00:00<?, ?it/s]/usr/local/lib/python3.8/dist-packages/keras/engine/functional.py:637: UserWarning: Input dict contained keys ['user_avg_rating', 'user_std_rating', 'user_rating_count'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


4/4 [==============================] - 0s 41ms/step


  0%|          | 1/2963 [00:04<3:59:09,  4.84s/it]

  1/474 [..............................] - ETA: 48s

/usr/local/lib/python3.8/dist-packages/keras/engine/functional.py:637: UserWarning: Input dict contained keys ['user_avg_rating', 'user_std_rating', 'user_rating_count'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


...

/usr/local/lib/python3.8/dist-packages/keras/engine/functional.py:637: UserWarning: Input dict contained keys ['user_avg_rating', 'user_std_rating', 'user_rating_count'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


5/5 [==============================] - 0s 47ms/step


100%|█████████▉| 2962/2963 [4:18:05<00:04,  4.73s/it]/usr/local/lib/python3.8/dist-packages/keras/engine/functional.py:637: UserWarning: Input dict contained keys ['user_avg_rating', 'user_std_rating', 'user_rating_count'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


4/4 [==============================] - 0s 43ms/step


100%|██████████| 2963/2963 [4:18:09<00:00,  5.23s/it]


In [17]:
MR = pd.DataFrame(movie_recommendations)
all_test_users = top_50_recs['userid'].unique().tolist()
pd_users = pd.DataFrame(all_test_users, columns=['userid'])
concat = pd.concat([pd_users, MR], axis = 1)
concat

,userid,0,1,2,3,4,5,6,7,8,...,10,11,12,13,14,15,16,17,18,19
0,55,206,519,272,198,168,61,637,307,936,...,222,643,774,977,591,62,1214,530,110,510
1,133,40,472,73,165,617,121,376,215,106,...,577,410,216,101,398,421,230,537,110,146
2,136,233,226,61,491,591,597,381,668,179,...,1214,566,162,510,472,215,537,101,376,221
3,213,262,285,221,146,101,216,154,165,215,...,40,106,472,121,230,73,398,62,190,491
4,539,281,423,504,171,123,129,1263,517,303,...,258,307,510,179,146,491,1214,578,566,977
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2958,282663,40,472,73,398,216,230,421,106,376,...,537,410,577,146,110,215,165,121,101,221
2959,282836,389,256,76,307,285,566,381,637,215,...,190,221,165,101,42,977,1214,572,154,510
2960,282999,674,174,592,476,770,569,390,658,882,...,171,517,1263,392,1214,258,206,40,222,61
2961,283047,40,472,73,617,121,106,215,146,101,...,165,410,216,376,398,577,230,537,110,421


In [18]:
all_test_users = top_50_recs['userid'].unique().tolist()

def get_top20_by_embed(userid):  
    return concat.loc[concat['userid'] == str(userid)][0:].values.tolist()[0][1:]

userid_pd = []
movieid_pd = []
for i in range(len(all_test_users)):
    user = all_test_users[i]
    top_20 = get_top20_by_embed(user)
    for j in range(20):
        userid_pd.append(int(user))
        movieid_pd.append(top_20[j])


In [19]:
userid_pd = pd.DataFrame(userid_pd, columns=['userid'])
movieid_pd = pd.DataFrame(movieid_pd, columns=['movieid'])
pd = pd.concat([userid_pd,movieid_pd], axis=1)
pd.to_csv('top_20_update_2.csv')